In [1]:
import torch
import torch.nn as nn


<image src="im1.png">

In [19]:
matr = torch.Tensor(
    [[[1, 2, 1],
      [2, 4, 2],
      [1, 2, 1]],
     [[-1, 0, 1],
     [0, 0, 0],
     [1, 0, -1]],
     [[0, 1, 0],
     [1, 2, 1],
     [0, 1, 0]]])

kernel = torch.Tensor(
    [[[1, 0],
      [0, 1]],
     [[0, -1],
      [-1, 0]],
     [[1, -1],
        [0, 0]]
     ]).view(1, 3,2,2)


res = nn.functional.conv2d(matr, kernel, padding=1, dilation=2)


print(res)


tensor([[[ 4.,  2.,  0.],
         [ 1.,  0.,  3.],
         [-2.,  2.,  6.]]])


<image src="im2.png">


In [7]:
from dataclasses import dataclass


@dataclass
class LayerInfo():
    kernelSize: int
    padding: int
    stride: int
    dilation: int


def getLayers() -> list[LayerInfo]:
    layer1 = LayerInfo(7, 3, 1, 1)
    layer2 = LayerInfo(3, 1, 2, 1)
    layer3 = LayerInfo(2, 0, 2, 1)
    layer4 = LayerInfo(3, 100500, 1, 2)
    layer5 = LayerInfo(3, 28, 3, 1)
    return list(locals().values())


stridesM = 1
receptiveField = 1
for id, layer in enumerate(getLayers()):
    stridesM *= layer.stride
    receptiveField += stridesM * ((layer.dilation * (layer.kernelSize-1)))
print(receptiveField)

from torch_receptive_field import receptive_field

model = nn.Sequential(
    nn.Conv2d(1, 1, 7, 1, 0, 1),
    nn.Conv2d(1, 1, 3, 2, 0, 1),
    nn.MaxPool2d(2, 2, 0),
    nn.Conv2d(1, 1, 3, 1, 0, 2),
    nn.MaxPool2d(3, 3, 0)
)
model.cuda()
dict = receptive_field(model, input_size=(1, 1000, 1000))

55
------------------------------------------------------------------------------
        Layer (type)    map size      start       jump receptive_field 
        0             [1000, 1000]        0.5        1.0             1.0 
        1             [994, 994]        3.5        1.0             7.0 
        2             [496, 496]        4.5        2.0             9.0 
        3             [248, 248]        5.5        4.0            11.0 
        4             [244, 244]        9.5        4.0            27.0 
        5               [81, 81]       13.5       12.0            35.0 


<img src="im3.png">

In [34]:
image = torch.empty((1, 224, 224))

conveyor = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=1,
              kernel_size=7, stride=2, padding=3),
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1),
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3,
              padding=1, stride=2, dilation=3)
)

print(conveyor(image).shape)


torch.Size([1, 26, 26])
